## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = ['/kaggle/input/116-googleq-a-train-bert-base-uncased-3f-lrwarmup1/model_fold_1.h5']
print('Models to predict:', model_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/116-googleq-a-train-bert-base-uncased-3f-lrwarmup1/model_fold_1.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.958624,0.655708,0.193221,0.419030,0.588321,0.686957,0.752025,0.669664,0.759817,...,0.956452,0.921800,0.528050,0.969696,0.949610,0.830215,0.043817,0.034526,0.828459,0.915883
1,46,0.918053,0.577097,0.008948,0.727469,0.755758,0.923140,0.551179,0.486852,0.087161,...,0.712938,0.961174,0.653471,0.976591,0.980975,0.855393,0.936123,0.091592,0.065627,0.899039
2,70,0.926876,0.656334,0.046836,0.747974,0.862198,0.852992,0.651963,0.532505,0.153808,...,0.925193,0.915031,0.676148,0.974095,0.964834,0.864165,0.116761,0.118559,0.836929,0.906627
3,132,0.872878,0.359536,0.005715,0.606214,0.769222,0.896272,0.573656,0.462918,0.084106,...,0.665116,0.943982,0.679978,0.976341,0.986232,0.872931,0.906269,0.167415,0.248718,0.893371
4,200,0.960782,0.353344,0.077213,0.752584,0.803052,0.830341,0.711915,0.581805,0.107091,...,0.704203,0.867878,0.625510,0.973303,0.964244,0.815898,0.115484,0.060952,0.612921,0.914389


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.889926,0.576761,0.030357,0.704398,0.806810,0.839751,0.589477,0.481747,0.239474,...,0.799587,0.930735,0.671776,0.962724,0.971286,0.868674,0.534934,0.141903,0.501739,0.913040
std,2812.670060,0.060687,0.166907,0.063684,0.138400,0.118305,0.152414,0.064693,0.100918,0.234126,...,0.102162,0.032952,0.061239,0.015926,0.016167,0.047213,0.344608,0.081494,0.309553,0.027122
min,39.000000,0.608664,0.245653,0.002186,0.189410,0.288289,0.107813,0.435949,0.280235,0.004013,...,0.488641,0.708982,0.485916,0.861075,0.845360,0.700604,0.002076,0.003709,0.004831,0.771997
25%,2572.000000,0.853050,0.427691,0.005888,0.607158,0.750385,0.813293,0.545867,0.410781,0.060733,...,0.713132,0.913797,0.635054,0.953590,0.964502,0.839251,0.171999,0.084078,0.212491,0.899158
50%,5093.000000,0.897247,0.566621,0.009429,0.716689,0.815652,0.896886,0.583554,0.455194,0.140667,...,0.808110,0.937740,0.673581,0.966575,0.975025,0.874869,0.609877,0.133680,0.515897,0.915986
75%,7482.000000,0.938647,0.719486,0.020263,0.809067,0.896075,0.933992,0.630259,0.539892,0.352048,...,0.892017,0.953686,0.714898,0.974111,0.982263,0.903893,0.861173,0.189330,0.774127,0.931531
max,9640.000000,0.988575,0.925343,0.641130,0.980593,0.987094,0.986006,0.787027,0.773397,0.900235,...,0.966042,0.984675,0.845378,0.992966,0.992639,0.971708,0.979192,0.405229,0.995025,0.971184
